In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import t
import seaborn as sns
import matplotlib.pyplot as plt

from src.statistics.stats import compare_score

In [ ]:
df = pd.read_csv("../results/radiomics/rkf_results/light.csv")
df_combat = pd.read_csv("../results/radiomics/rkf_results/light_combat.csv")
df_fusion = pd.read_csv("../results/radiomics/rkf_results/light_fusion.csv")
df_fusion["modality"] = "PET/CT"
df_dummy = pd.read_csv("../results/radiomics/rkf_results/dummy.csv")
df_suvmax = pd.read_csv("../results/radiomics/rkf_results/suvmax.csv")

In [ ]:
df_combat[(df_combat["modality"] == "CT") & (df_combat["voi"] == "GTV_N")].mean()

In [ ]:
df = pd.concat([df, df_fusion], axis=0)

In [ ]:
df[(df["modality"] == "CT") & (df["voi"] == "GTV_N")].mean()


In [ ]:
df_dummy["modality"] = "random"
df_dummy["voi"] = df_dummy["strategy"].map(lambda x: "most_frequent" if x=="prior" else x)

In [ ]:
df_suvmax["modality"] = "SUVmax"
df_suvmax["voi"] = df_suvmax["strategy"]

In [ ]:
df_tot = pd.concat([df, df_dummy[df_dummy["voi"]!="most_frequent"], df_suvmax], axis=0)

In [ ]:
df_tot

In [ ]:
sns.set(rc={"figure.figsize":(18, 18)}) #width=8, height=4
ax = sns.catplot(x="modality", y="roc_auc", hue="voi", data=df, kind="box", palette="magma", width=0.5)
plt.savefig("/home/valentin/python_wkspce/plc_segmentation/reports/figures/results_ptct.png")

In [ ]:
df[(df["modality"]=="PET/CT")&(df["voi"]=="GTV_L")].mean()

In [ ]:
sns.set(rc={"figure.figsize":(8, 9)}) #width=8, height=4
ax = sns.catplot(x="modality", y="roc_auc", hue="voi", data=df_tot, kind="box", palette="magma", width=1.0)

In [ ]:
df_tot[df_tot["voi"]=="random_features"].mean()

In [ ]:
ax = sns.barplot(x="modality", y="roc_auc", hue="voi", data=df, ci="sd")

In [ ]:
df_dummy

In [ ]:
s1 = df.loc[(df["modality"]=="PT") & (df["voi"]=="GTV_N"), "roc_auc"].values
# s2 = df.loc[(df["modality"]=="CT") & (df["voi"]=="GTV_L"), "roc_auc"].values

In [ ]:
s1 = df_dummy.loc[df_dummy["strategy"]=="center", "roc_auc"].values
s2 = df_dummy.loc[df_dummy["strategy"]=="random_features", "roc_auc"].values

In [ ]:
s = compare_score(s1, s2, 95, 11)

In [ ]:
print(s)

In [ ]:
s["p_value"] + s["proba M1 > M2"]